# 1. Inference Notebook, follow up [Training Notebook](https://www.kaggle.com/mlneo07/swin-transformer-fasterrcnn-training) 

# **Install Offline MMDetection**

In [ ]:
!pip install '../input/pytorch-190/torch-1.9.0+cu111-cp37-cp37m-linux_x86_64.whl'

In [ ]:
!pip install '/kaggle/input/mmdetectionv2140/addict-2.4.0-py3-none-any.whl' 
!pip install '/kaggle/input/mmdetectionv2140/yapf-0.31.0-py2.py3-none-any.whl' 
!pip install '/kaggle/input/mmdetectionv2140/terminal-0.4.0-py3-none-any.whl'
!pip install '/kaggle/input/mmdetectionv2140/terminaltables-3.1.0-py3-none-any.whl'
!pip install '/kaggle/input/mmdetection-v217/mmdetection/mmcv_full-1.3.x-py2.py3-none-any/mmcv_full-1.3.16-cp37-cp37m-manylinux1_x86_64.whl'
!pip install '/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2'
!pip install '/kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/mmpycocotools-12.0.3'

!rm -rf mmdetection

!cp -r /kaggle/input/mmdetection-v217/mmdetection/mmdetection-2.18.0 /kaggle/working/
!mv /kaggle/working/mmdetection-2.18.0 /kaggle/working/mmdetection
%cd /kaggle/working/mmdetection
!pip install -e .

%cd ..

# !rm -rf mmdetection
# !git clone https://github.com/open-mmlab/mmdetection.git /kaggle/working/mmdetection

In [ ]:
# !pip install openmim
# !mim install mmdet

# **Import Libraries**

In [ ]:
import sys, os
sys.path.append('./mmdetection')
sys.path.append('../input/tensorflow-great-barrier-reef/greatbarrierreef')
import numpy as np
import mmdet
import mmcv
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import inference_detector, init_detector, show_result_pyplot, set_random_seed

In [ ]:
seed = 42
set_random_seed(seed, deterministic=False)

# **Model**

In [ ]:


%%writefile ./mmdetection/configs/swin/TFGBR_swin_base_faster_rcnn_fp16.py

_base_ = [
    '../_base_/models/cascade_rcnn_r50_fpn.py',
    '../_base_/datasets/coco_detection.py',
    '../_base_/schedules/schedule_1x.py', '../_base_/default_runtime.py'
]
pretrained = 'https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_small_patch4_window7_224.pth'
model = dict(
    backbone=dict(
        _delete_=True,
        type='SwinTransformer',
        embed_dims=96,
        depths=[2, 2, 18, 2],
        num_heads=[3, 6, 12, 24],
        window_size=7,
        mlp_ratio=4,
        qkv_bias=True,
        qk_scale=None,
        drop_rate=0.,
        attn_drop_rate=0.,
        drop_path_rate=0.3,
        patch_norm=True,
        out_indices=(0, 1, 2, 3),
        with_cp=False,
        convert_weights=True,
        init_cfg=dict(type='Pretrained', checkpoint=pretrained)),
    neck=dict(in_channels=[96, 192, 384, 768]),
    roi_head=dict(
        type='CascadeRoIHead',
        num_stages=3,
        stage_loss_weights=[1, 0.5, 0.25],
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        bbox_head=[
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=1,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.1, 0.1, 0.2, 0.2]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                reg_decoded_bbox=True,
                loss_bbox=dict(type='GIoULoss', loss_weight=10.0)),
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=1,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.05, 0.05, 0.1, 0.1]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                reg_decoded_bbox=True,
                loss_bbox=dict(type='GIoULoss', loss_weight=10.0)),
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=1,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.033, 0.033, 0.067, 0.067]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                reg_decoded_bbox=True,
                loss_bbox=dict(type='GIoULoss', loss_weight=10.0))
        ]))

optimizer = dict(
    _delete_=True,
    type='AdamW',
    lr=0.0004,
    betas=(0.9, 0.999),
    weight_decay=0.05,
    paramwise_cfg=dict(
        custom_keys={
            'absolute_pos_embed': dict(decay_mult=0.),
            'relative_position_bias_table': dict(decay_mult=0.),
            'norm': dict(decay_mult=0.)
        }))
lr_config = dict(warmup_iters=500, step=[8, 11])
runner = dict(max_epochs=14)

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./mmdetection/configs/swin/TFGBR_swin_base_faster_rcnn_fp16.py')

In [ ]:
%%writefile labels.txt 
cots

In [ ]:
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile',to_float32=True),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='AutoAugment',
        policies=[[
            dict(
                type='Resize',
                img_scale=[(480, 1333), (512, 1333), (544, 1333), (576, 1333),
                           (608, 1333), (640, 1333), (672, 1333), (704, 1333),
                           (736, 1333), (768, 1333), (800, 1333)],
                multiscale_mode='value',
                keep_ratio=True)
        ],
                  [
                      dict(
                          type='Resize',
                          img_scale=[(400, 1333), (500, 1333), (600, 1333)],
                          multiscale_mode='value',
                          keep_ratio=True),
                      dict(
                          type='RandomCrop',
                          crop_type='absolute_range',
                          crop_size=(384, 600),
                          allow_negative_crop=True),
                      dict(
                          type='Resize',
                          img_scale=[(480, 1333), (512, 1333), (544, 1333),
                                     (576, 1333), (608, 1333), (640, 1333),
                                     (672, 1333), (704, 1333), (736, 1333),
                                     (768, 1333), (800, 1333)],
                          multiscale_mode='value',
                          override=True,
                          keep_ratio=True),
                      dict(
                            type='PhotoMetricDistortion',
                            brightness_delta=32,
                            contrast_range=(0.5, 1.5),
                            saturation_range=(0.5, 1.5),
                            hue_delta=18),
                    dict(
                            type='MinIoURandomCrop',
                            min_ious=(0.4, 0.5, 0.6, 0.7, 0.8, 0.9),
                            min_crop_size=0.3),
                    dict(
                            type='CutOut',
                            n_holes=(5, 10),
                            cutout_shape=[(4, 4), (4, 8), (8, 4), (8, 8),
                                          (16, 32), (32, 16), (32, 32),
                                          (32, 48), (48, 32), (48, 48)]
                            )
                  ]]),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]

test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img'])
        ])
]


In [ ]:
cfg.classes = '/kaggle/working/labels.txt'
cfg.work_dir = '/kaggle/working/model_output'
cfg.data_root = '/kaggle/working'

# cfg.model.roi_head.bbox_head.num_classes = 1

cfg.data.test.type = 'CocoDataset'
cfg.data.test.classes = 'labels.txt'
cfg.data.test.data_root = '/kaggle/working'
cfg.data.test.ann_file = '../input/tfreef-coco-dataset-notebook/val_dataset.json'
cfg.data.test.img_prefix = ''

cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = '/kaggle/working'
cfg.data.train.ann_file = '../input/tfreef-coco-dataset-notebook/train_dataset.json'
cfg.data.train.img_prefix = ''
cfg.data.train.classes = 'labels.txt'

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = '/kaggle/working'
cfg.data.val.ann_file = '../input/tfreef-coco-dataset-notebook/val_dataset.json'
cfg.data.val.img_prefix = ''
cfg.data.val.classes = 'labels.txt'


cfg.train_pipeline = train_pipeline
cfg.val_pipeline = test_pipeline
cfg.test_pipeline = test_pipeline

cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.val.pipeline = cfg.val_pipeline

# cfg.optimizer.lr = 0.02 / 8
# cfg.lr_config = dict(
#     policy='CosineAnnealing', 
#     by_epoch=False,
#     warmup='linear', 
#     warmup_iters= 125, 
#     warmup_ratio= 1/12,
#     min_lr=1e-07)

cfg.runner.max_epochs = 12
cfg.evaluation.save_best='auto'

cfg.seed = seed
cfg.gpu_ids = range(1)
cfg.fp16 = dict(loss_scale=512.)
meta = dict()
meta['config'] = cfg.pretty_text


# **Inference**

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

In [ ]:
import greatbarrierreef

env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [ ]:
model = init_detector(cfg, '../input/mmdetection-swin-transformer-fasterrcnn-training/model_output/epoch_14.pth')

In [ ]:
#################################### faster rcnn ############################################
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
# checkpoint = load_checkpoint(model, WEIGHTS_FILE, map_location='cpu')

# model.CLASSES = dataset.CLASSES

# model = MMDataParallel(model, device_ids=[0])
# outputs = single_gpu_test(model, data_loader, False, None, 0.5)

results = []

for (pixel_array, sample_prediction_df) in iter_test:
    result = inference_detector(model, pixel_array[:, :, ::-1])
#     show_result_pyplot(model, pixel_array[:, :, ::-1], result)    
    boxes = result[0][:, :4]
    scores = result[0][:, 4]

    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]

    
    sample_prediction_df['annotations'] = format_prediction_string(boxes, scores)
    
    env.predict(sample_prediction_df)

In [ ]:
import shutil
shutil.rmtree('/kaggle/working/mmdetection')